# Tutorium 2

## Mathematisches Modell

**Zielfunktion**

\begin{equation}
	minimiere\ \ Z = \sum^{I}_{i=1} f_{i} \cdot y_{i} + \sum^{I}_{i=1} \sum^{J}_{j=1} c_{ij} \cdot x_{ij}
\end{equation}

**unter den Nebenbedingungen**

\begin{align}
&& \sum^{I}_{i=1} x_{ij} &= d_{j} && \forall j \in J \\[5pt]
&& \sum^{J}_{j=1} x_{ij} &\leq b_{i} \cdot y_{i} && \forall i \in I \\[10pt]
&& x_{ij} &\geq 0 && \forall i \in I, \forall j \in J \\[5pt]
&& y_{i} &\in \left\{ 0, 1 \right\} &&
\end{align}


## Aufgabe 3b)
Lesen Sie aus den csv-Dateien die erforderlichen Daten ein inkl. der Namen der Absatzorte und Standorte.

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import csv

### Absatzorte.csv

| Ort       | Bedarf | 
|:----------|--------| 
| Frankfurt | 200    | 
| Berlin    | 250    | 
| München   | 150    | 
| Köln      | 300    | 
| Hamburg   | 100    | 

In [ ]:
d=[]
ao=[]
with open("Absatzorte.csv", encoding="utf-8") as csv_file:
     csv_reader = csv.DictReader(csv_file)
     for row in csv_reader:
          d.append(int(row["Bedarf"]))
          ao.append(row["Ort"])

### Standorte.csv

| Ort        | Kapazität | Fixkosten | 
|:-----------|-----------|-----------| 
| Dresden    | 400       | 60000     | 
| Bremen     | 350       | 60000     | 
| Düsseldorf | 600       | 80000     | 

In [ ]:
b=[]
f=[]
so=[]
with open("Standorte.csv", encoding="utf-8") as csv_file:
     csv_reader = csv.DictReader(csv_file)
     for row in csv_reader:
          b.append(int(row["Kapazität"]))
          f.append(int(row["Fixkosten"]))
          so.append(row["Ort"])

Hinweis: Zum Einlesen von der Distanz und der Fahrtdauer empfiehlt sich ein Einlesen mit `csv.reader()`, da nicht mit den Spaltennamen gearbeitet werden muss.

### Dauer.csv

| Orte       | Frankfurt | Berlin | München | Köln | Hamburg | 
|:-----------|-----------|--------|---------|------|---------| 
| Dresden    | 4         | 2      | 5       | 6    | 5       | 
| Bremen     | 4         | 4      | 8       | 3    | 2       | 
| Düsseldorf | 2         | 6      | 7       | 1    | 5       | 

In [ ]:
dur=[]
with open("Dauer.csv", encoding="utf-8") as csv_file:
     csv_reader = csv.reader(csv_file)
     next(csv_reader)
     for row in csv_reader:
          rowAsInt = [int(item) for item in row[1:]]
          dur.append(rowAsInt)

### Enfernung.csv

| Orte       | Frankfurt | Berlin | München | Köln | Hamburg | 
|:-----------|-----------|--------|---------|------|---------| 
| Dresden    | 460       | 190    | 460     | 570  | 500     | 
| Bremen     | 440       | 400    | 750     | 320  | 130     | 
| Düsseldorf | 230       | 560    | 610     | 40   | 410     | 


In [ ]:
dist=[]
with open("Entfernung.csv", encoding="utf-8") as csv_file:
     csv_reader = csv.reader(csv_file)
     next(csv_reader)
     for row in csv_reader:
          rowAsInt = [int(item) for item in row[1:]]
          dist.append(rowAsInt)

## Aufgabe 3c)
Da Ihnen die Werte für die Transportkosten nicht gegeben sind, berechnen Sie diese aus den vorhandenen Daten Dauer und Entfernung. Gehen Sie von einem km-Kostensatz von 0,75 GE/km und einem Stundenlohn von 45 GE aus.

### Definition der Mengen

In [ ]:
I_max = len(b)
J_max = len(d)

I = range(I_max)
J = range(J_max)

### Berechnung der Kosten

In [ ]:
c = []
for i in I:
     newRow = []
     for j in J:
          newRow.append(costperH * dur[i][j] + costPerKm * dist[i][j])
     c.append(newRow)

## Aufgabe 3d)
Lassen Sie sich zur Kontrolle jede Zeile der berechneten Kostenmatrix in der Konsole ausgeben und lösen Sie das Problem.

Initialisierung des Modells:

In [ ]:
m = gp.Model()

### Initialisierung der Variablen:

Variante - Wiederholter Aufruf von `model.addVar()`

In [ ]:
x = {}
for i in I:
     for j in J:
          x[i,j] = m.addVar(vtype=GRB.CONTINUOUS, name="x_"+str(i)+str(j))

In [ ]:
y = {}
for i in I:
     y[i] = m.addVar(vtype=GRB.BINARY, name="y_"+str(i))

Variante - Aufruf von `model.addVars()` 

In [ ]:
x = m.addVars(I_max, J_max, vtype=GRB.CONTINUOUS, name="x")

In [ ]:
y = m.addVars(I_max, vtype=GRB.BINARY, name="y")

Definition der Zielfunktion

In [ ]:
m.setObjective(gp.quicksum(f[i] * y[i] for i in I) + gp.quicksum(c[i][j] * x[i,j] for j in J for i in I), GRB.MINIMIZE)

### Hinzufügen der Nebenbedingungen

Variante - Wiederholter Aufruf von `model.addConstr()`

In [ ]:
for j in J:
     m.addConstr(gp.quicksum(x[i,j] for i in I) == d[j], "nb_" + str(j))

In [ ]:
for i in I:
     m.addConstr(gp.quicksum(x[i,j] for j in J) <= b[i] * y[i], "kb_" + str(i))

Variante - Aufruf von `model.addConstrs()`

In [ ]:
m.addConstrs((x.sum("*", j) == d[j] for j in J), name="nb")

In [ ]:
m.addConstrs((x.sum(i, "*") == b[i] * y[i] for i in I), name="kb")

Hinweis: `var.sum()` funktioniert nur, wenn vorher `var` mit `model.addVars()` erzeugt wurde, da `model.addVars()` ein Tupledict zurückgibt und `var.sum()` nur für Tupledicts definiert ist. 

### Optimierung:

In [ ]:
m.optimize()

Ergebnisausgabe:

In [ ]:
m.printAttr(GRB.Attr.ObjVal)

In [ ]:
m.printAttr(GRB.Attr.X)

## Aufgabe 2d)
Angenommen das Unternehmen besitzt keine Lieferverpflichtung und erhält pro Transport eine Entschädigung von 350 GE. Welche Transporte würden dann ausgeführt
werden, wo werden Standorte gebaut und wie hoch ist der Gewinn? Ändern sie hierfür die erforderlichen Nebenbedingungen und die Zielfunktion und optimieren Sie erneut.

Keine Lieferverpflichtung:
Nachfrage pro Standort muss nicht mehr vollständig gedeckt sein.

In [ ]:
for j in J:
     m.addConstr(gp.quicksum(x[i,j] for i in I) <= d[j], "nb_" + str(j))

Transportentschädigung: Gewinn wird mit variablen Transportkosten verrechnet.

In [ ]:
g = 350
gewinn = gp.quicksum((g - c[i][j]) * x[i,j] for j in J for i in I)
fixeKosten = gp.quicksum(f[i] * y[i] for i in I)
m.setObjective(gewinn - fixeKosten, GRB.MAXIMIZE)